* 文字列データを表形式データに変換するコード
* 出力されたコードをコードモデルに組み込むコード

In [3]:
import sys
sys.path.append('..')

from src.ibldataset.psuedodata import jinja_rendering, df_to_string

from datasets import load_dataset

dataset_name = 'ibl-regression-ver2-linear'

In [7]:
code = """
if row["B"] < 18:
    y = 8*row["C"]
else:
    y = 4*row["C"]
"""

In [ ]:
def predict(x):
    df = x.copy()
    output = []
    for index, row in df.iterrows():
        y = 2*row["A"] + 0*row["B"] + 10*row["C"]
        output.append(y)
    return np.array(output)

In [8]:
code_model = jinja_rendering(code)
print(code_model)

def predict(x):
    df = x.copy()
    output = []
    for index, row in df.iterrows():
        
        if row["B"] < 18:
            y = 8*row["C"]
        else:
            y = 4*row["C"]
        
        output.append(y)
    return np.array(output)


In [9]:
test_dataset = load_dataset(f"fuyu-quant/{dataset_name}", split='test')

In [10]:
i = 1
data_str = test_dataset[i]['instruction']

In [ ]:
def ibl_output(model, input):
    prompt = f"### Instruction: {input}\n\n### Response: "

    # 推論の実行
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100
            )
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    output = output.split("### Response: ")[-1]
    return output

In [11]:
import pandas as pd
import numpy as np

def str_to_df(data_str):
    lines = data_str.split('\n')
    data = [line.split() for line in lines]
    df = pd.DataFrame(data[1:], columns=data[0])
    df = df.astype(int)
    return df

def code_model_pred(df, code_model):
    df = df.drop(['y'], axis=1)
    local_vars = {}
    exec(code_model, globals(), local_vars)
    df['y'] = local_vars['predict'](df)
    return df

In [12]:
df = str_to_df(data_str)
pred_df = code_model_pred(df, code_model)
str_df = pred_df.to_string(index=False)
str_df

In [16]:
import random

In [17]:
import random

def linear_model():
    processing = ''

    coef_A = random.randint(0, 10)
    coef_B = random.randint(0, 10)
    coef_C = random.randint(0, 10)
    processing += f'y = {coef_A}*row["A"] + {coef_B}*row["B"] + {coef_C}*row["C"]'

    return processing, coef_A, coef_B, coef_C


def find_solutions(coef_A, coef_B, coef_C, total=1500):
    solutions = []
    for c in range(total // 5 + 1):
        for b in range((total - 5 * c) // 3 + 1):
            a = (total - 5 * c - 3 * b) // 2
            if coef_A * a + coef_B * b + coef_C* c == total:
                solutions.append([a, b, c, total])
    return solutions

def generate_data(num_rows):
    processing, coef_A, coef_B, coef_C = linear_model()
    sample_size = num_rows / 2
    data_1000 = find_solutions(coef_A, coef_B, coef_C, total=1500)
    data_1000 = random.sample(data_1000, sample_size)
    data_1500 = find_solutions(coef_A, coef_B, coef_C, total=1000)
    data_1500 = random.sample(data_1500, sample_size)
    data = data_1000 + data_1500
    random.shuffle(data)
    df = pd.DataFrame(data, columns=['A', 'B', 'C', 'y'])
    str_df = df.to_string(index=False)
    return processing, str_df

In [20]:
solution_list = find_solutions()
sample_size = 50
solutions = random.sample(solution_list, sample_size)

In [21]:
solutions

[[242, 162, 6],
 [97, 162, 64],
 [51, 121, 107],
 [244, 79, 55],
 [212, 57, 81],
 [280, 120, 16],
 [357, 37, 35],
 [218, 8, 108],
 [224, 14, 102],
 [62, 212, 48],
 [27, 312, 2],
 [161, 211, 9],
 [76, 226, 34],
 [178, 123, 55],
 [270, 10, 86],
 [143, 33, 123],
 [50, 265, 21],
 [393, 13, 35],
 [344, 39, 39],
 [114, 109, 89],
 [250, 15, 91],
 [15, 45, 167],
 [125, 65, 111],
 [92, 242, 18],
 [350, 75, 15],
 [168, 3, 131],
 [176, 71, 87],
 [221, 161, 15],
 [335, 70, 24],
 [52, 297, 1],
 [53, 128, 102],
 [190, 25, 109],
 [156, 1, 137],
 [270, 25, 77],
 [184, 79, 79],
 [287, 92, 30],
 [233, 93, 51],
 [82, 52, 136],
 [371, 56, 18],
 [173, 103, 69],
 [209, 189, 3],
 [216, 106, 50],
 [56, 191, 63],
 [85, 105, 103],
 [63, 198, 56],
 [194, 149, 33],
 [300, 60, 44],
 [37, 22, 172],
 [318, 78, 26],
 [38, 308, 0]]